# Couche bronze
  Objectif: Collecte des différentes sources de données

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col

spark = SparkSession.builder \
    .appName("BronzeLayer") \
    .getOrCreate()

## Création du parquet contenant les données du csv

In [12]:
df = spark.read.csv("../Source/data.csv", header=True, inferSchema=True)

df.write.parquet("data/csv", mode="overwrite")

## Création du parquet contenant les données streamer avec kafka

In [ ]:
kafka_df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "your_kafka_topic") \
    .option("startingOffsets", "latest") \
    .load()

# Extract JSON string from Kafka 'value' column
json_df = kafka_df.selectExpr("CAST(value AS STRING) as json_str")

# Parse JSON string into columns
parsed_df = json_df.select(from_json(col("json_str"), json_schema).alias("data")).select("data.*")

query = parsed_df.writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "data/stream") \
    .option("checkpointLocation", "checkpoint") \
    .start()

query.awaitTermination()

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.